In [73]:
# default_exp engine

# longtail

> API details.

In [74]:
#hide
from nbdev.showdoc import *

In [75]:
#export
import functools
import time
from collections import namedtuple
import csv
import pandas as pd
from pathlib import Path

In [76]:
#hide
def waste_some_time(num_times: int):"""

    :param num_times: The number of times a function is called
    :return:  None
    """
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [2]:
#hide
!nbdev_build_lib

zsh:1: command not found: nbdev_build_lib


In [3]:
#export


class BlockTracker(object):
    
    def register(self, func):
        """Print the function signature and return value"""
        @functools.wraps(func)
        def wrapper_debug(*args, **kwargs):

            if "__kwdefaults__" in dir(func):
                d = func.__kwdefaults__
                if d is not None and "context" in d and d["context"] is None:
                    d["context"] = self

            args_repr = [repr(a) for a in args]                      # 1
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
            signature = ", ".join(args_repr + kwargs_repr)           # 3
            start_time = time.perf_counter()    # 1
            value = str(func(*args, **kwargs))
            end_time = time.perf_counter()      # 2
            run_time = end_time - start_time    # 3
            self.command_list.append(self.BlockCommand(name=func.__name__,
                                                       signature=signature,
                                                       value=value,
                                                       start=round(start_time, 2),
                                                       total=round(run_time, 9),
                                                       batch=self.batch,
                                                       stanza=self.stanza
                                                       ))

            if "__kwdefaults__" in dir(func):
                d = func.__kwdefaults__
                if d is not None and "context" in d and d["context"] == self:
                    d["context"] = None

            return value
        return wrapper_debug

    def __init__(self, file_name, stanza):
        self.path = file_name
        self.command_list = []
        self.code_list = []
        self.stanza = stanza
        self.start_time =  time.perf_counter()
        self.batch = str(self.start_time)
        self.BlockCommand = namedtuple("BlockCommand", "name signature value start total batch stanza")

    def data(self):
        return pd.read_csv(self.path)

    def __enter__(self):
        self.start_time = time.perf_counter()
        return self.register

    def __exit__(self, type, value, traceback):
        self.start_time = time.perf_counter()

        for x in self.code_list:
            x.name(x.signature)

        with Path(self.path).open('a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(self.BlockCommand._fields)
            writer.writerows(self.command_list)

In [79]:
#hide
with BlockTracker("./block.txt",stanza="Mid file 2") as f:
    f(print)("hello")
    f(waste_some_time)(300)
    f(print)("world")
    f(waste_some_time)(100)


hello
world


In [3]:
#hide
lastWrapper = ""

def capture(*args, context:BlockTracker = None):

    global lastWrapper
    if not context is None:
        lastWrapper = context
    for x in args:
        print(x)


NameError: name 'BlockTracker' is not defined

In [81]:
#hide
with BlockTracker("./data/block.txt",stanza="Mid file 2") as f:
    f(capture)("hello","world")
    f(print)("human fish")
    f(waste_some_time)(100)
lastWrapper.data().head()

hello
world
human fish


,name,signature,value,start,total,batch,stanza
0,capture,"'hello', 'world'",None,4860.93,0.000340,4860.931309,Mid file 2
1,print,'human fish',None,4860.93,0.000021,4860.931309,Mid file 2
2,waste_some_time,100,None,4860.93,0.264946,4860.931309,Mid file 2
